# Part 9: Random Forest regression model

The potency of the novel generated compounds for the BACE1 receptor will be determined with machine learning models. In this project will two different models be compared; a random forest regression model and a neuron network model. These models will be trained and validated with the filtered ChEMBL bioactivity data from part 3. In this part will the random forest regression model be trained and validated, followed by using the model to predict the potency of the novel compounds.

Import required libraries

In [1]:
from pathlib import Path
from warnings import filterwarnings
import time

import pandas as pd
import numpy as np
from sklearn import svm, metrics, clone
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import auc, accuracy_score, recall_score
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.Chem import PandasTools

import sys

from teachopencadd.utils import seed_everything
SEED = 22
seed_everything(SEED)


Define paths

In [2]:
HERE = Path(_dh[-1])
DATA = HERE / "data"

Read filtered ChEMBL bioactivity data from part 3

In [3]:
chembl_df = pd.read_csv(DATA/"BACE_compounds.csv",
    index_col=0,
)

print("Shape of dataframe : ", chembl_df.shape)
chembl_df.head()

Shape of dataframe :  (6691, 7)


,molecule_chembl_id,IC50,units,smiles,pIC50,ROMol,graph
0,CHEMBL3969403,0.0002,nM,CC1(C)C(N)=N[C@](C)(c2cc(NC(=O)c3ccc(C#N)cn3)c...,12.698970,<rdkit.Chem.rdchem.Mol object at 0x7f85689dd580>,"[(x, [tensor([6, 0, 4, 5, 3, 0, 4, 0, 0]), ten..."
1,CHEMBL3937515,0.0009,nM,COc1cnc(C(=O)Nc2ccc(F)c([C@]3(C)CS(=O)(=O)C(C)...,12.045757,<rdkit.Chem.rdchem.Mol object at 0x7f85689dd620>,"[(x, [tensor([6, 0, 4, 5, 3, 0, 4, 0, 0]), ten..."
2,CHEMBL3949213,0.0010,nM,C[C@@]1(c2cc(NC(=O)c3ccc(C#N)cn3)ccc2F)CS(=O)(...,12.000000,<rdkit.Chem.rdchem.Mol object at 0x7f85689dd6c0>,"[(x, [tensor([6, 0, 4, 5, 3, 0, 4, 0, 0]), ten..."
3,CHEMBL3955051,0.0018,nM,CC1(C)C(N)=N[C@](C)(c2cc(NC(=O)c3cnc(C(F)F)cn3...,11.744727,<rdkit.Chem.rdchem.Mol object at 0x7f85689dd760>,"[(x, [tensor([6, 0, 4, 5, 3, 0, 4, 0, 0]), ten..."
4,CHEMBL3936264,0.0057,nM,C[C@@]1(c2cc(NC(=O)c3ccc(OC(F)F)cn3)ccc2F)CS(=...,11.244125,<rdkit.Chem.rdchem.Mol object at 0x7f85689dd800>,"[(x, [tensor([6, 0, 4, 5, 3, 0, 4, 0, 0]), ten..."


Keep only the needed columns

In [4]:
chembl_df = chembl_df[["molecule_chembl_id", "smiles", "pIC50"]]
chembl_df.head()

,molecule_chembl_id,smiles,graph,pIC50
0,CHEMBL3969403,CC1(C)C(N)=N[C@](C)(c2cc(NC(=O)c3ccc(C#N)cn3)c...,"[(x, [tensor([6, 0, 4, 5, 3, 0, 4, 0, 0]), ten...",12.698970
1,CHEMBL3937515,COc1cnc(C(=O)Nc2ccc(F)c([C@]3(C)CS(=O)(=O)C(C)...,"[(x, [tensor([6, 0, 4, 5, 3, 0, 4, 0, 0]), ten...",12.045757
2,CHEMBL3949213,C[C@@]1(c2cc(NC(=O)c3ccc(C#N)cn3)ccc2F)CS(=O)(...,"[(x, [tensor([6, 0, 4, 5, 3, 0, 4, 0, 0]), ten...",12.000000
3,CHEMBL3955051,CC1(C)C(N)=N[C@](C)(c2cc(NC(=O)c3cnc(C(F)F)cn3...,"[(x, [tensor([6, 0, 4, 5, 3, 0, 4, 0, 0]), ten...",11.744727
4,CHEMBL3936264,C[C@@]1(c2cc(NC(=O)c3ccc(OC(F)F)cn3)ccc2F)CS(=...,"[(x, [tensor([6, 0, 4, 5, 3, 0, 4, 0, 0]), ten...",11.244125


Add column for fingerprints

In [5]:
compound_df = chembl_df.copy()

Set the number of folds

In [6]:
N_FOLDS = 5

Fingerprints. The structures of the compounds will be transferred from SMILES notation to Morgan Fingerprints with a radius of 3, to explicitly describe the features of the compounds.

Define function to convert the smiles into fingerprints

In [ ]:
def smiles_to_fp(smiles, method="maccs", n_bits=2048):
    """
    Encode a molecule from a SMILES string into a fingerprint.

    Parameters
    ----------
    smiles : str
        The SMILES string defining the molecule.

    method : str
        The type of fingerprint to use. Default is MACCS keys.

    n_bits : int
        The length of the fingerprint.

    Returns
    -------
    array
        The fingerprint array.

    """

    # convert smiles to RDKit mol object
    mol = Chem.MolFromSmiles(smiles)

    if method == "maccs":
        return np.array(MACCSkeys.GenMACCSKeys(mol))
    if method == "morgan2":
        return np.array(GetMorganFingerprintAsBitVect(mol, 2, nBits=n_bits))
    if method == "morgan3":
        return np.array(GetMorganFingerprintAsBitVect(mol, 3, nBits=n_bits))
    else:
        # NBVAL_CHECK_OUTPUT
        print(f"Warning: Wrong method specified: {method}. Default will be used instead.")
        return np.array(MACCSkeys.GenMACCSKeys(mol))

[Moet dat niet later pas?]

Use Morgan fingerprint with radius 3

In [ ]:
compound_df["fp"] = compound_df["smiles"].apply(smiles_to_fp, args=("morgan3",))
compound_df.head()

Define function for machine learning model training and validation in a cross-validation loop

[Wat doet deze functie precies?]

In [ ]:
def crossvalidation_reg(ml_model, df, n_folds=5, verbose=False):
    """
    Machine learning model training and validation in a cross-validation loop.

    Parameters
    ----------
    ml_model: sklearn model object
        The machine learning model to train.
    df: pd.DataFrame
        Data set with SMILES and their associated activity labels.
    n_folds: int, optional
        Number of folds for cross-validation.
    verbose: bool, optional
        Performance measures are printed.

    Returns
    -------
    None

    """
    t0 = time.time()
    # Shuffle the indices for the k-fold cross-validation
    kf = KFold(n_splits=n_folds, shuffle=True)#, random_state=SEED)

    # Results for each of the cross-validation folds
    MAE_per_fold = []
    RMSE_per_fold = []

    # Loop over the folds
    for train_index, test_index in kf.split(df):
        # clone model -- we want a fresh copy per fold!
        fold_model = clone(ml_model)
        # Training

        # Convert the fingerprint and the label to a list
        train_x = df.iloc[train_index].fp.tolist()
        train_y = df.iloc[train_index].pIC50.tolist()

        # Fit the model
        fold_model.fit(train_x, train_y)

        # Testing

        # Convert the fingerprint and the label to a list
        test_x = df.iloc[test_index].fp.tolist()
        test_y = df.iloc[test_index].pIC50.tolist()

        test_results = fold_model.predict(test_x)
        # Prediction probability on test set
        from sklearn import metrics

        MAE_per_fold.append(metrics.mean_absolute_error(test_y, test_results))
        #print('Mean Squared Error (MSE):', metrics.mean_squared_error(test_y, test_results))
        RMSE_per_fold.append(np.sqrt(metrics.mean_squared_error(test_y, test_results)))

        #from sklearn.metrics import matthews_corrcoef


        #print(matthews_corrcoef(test_y, test_results))
        #mape = np.mean(np.abs((gt - pred) / np.abs(gt)))
        #print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
        #print('Accuracy:', round(100*(1 - mape), 2))
    return(MAE_per_fold,RMSE_per_fold,fold_model)

Random forest regressor model

The benefit of a regression model as opposed to a classification model is that it can not only predict if the new compound is active or inactive, but also the pIC50 value of the compound.

In [ ]:
models = [{"label": "Model_RF_reg", "model": RandomForestRegressor}]

Train model with RandomForestRegressor and show validation scores

In [ ]:
regressor = RandomForestRegressor()
MAE, RMSE,trained_model = crossvalidation_reg(regressor, compound_df, n_folds=N_FOLDS)

print(
f"Mean Absolute Error (MAE): {np.mean(MAE):.2f} \t"
f"and std : {np.std(MAE):.2f} \n"
f"Root Mean Square Error (RMSE): {np.mean(RMSE):.2f} \t"
f"and std : {np.std(RMSE):.2f} \n"
)


In [ ]:
compound_df.head()

To validate the performance of the model, the data was split into a train and test set. In this case this is done in a k-fold cross-validation. In this method, the data is split into k stratified folds. K separate models are trained, each with one of these folds as test data, and all other folds as training data. The k different calculated performances are then averaged for a more consistent performance overall. The errors between the predicted data and obtained experimental data are determined with a Mean Absolute Error (MAE) and a Root Mean Square Error (RMSE). 

[Herschrijven?]

[Zijn deze waarde oke? Hoe kunnen ze verbeterd worden? Below 0.6 quite decent?]

In [ ]:
print(trained_model)

This trained model will below be used to predict the pIC50 values of the novel, drug-like, generated compounds of part 8.

### Generated Compounds: predict pIC50 with the trained random forest regression model

Read the filtered, generated compound data of part 8

In [ ]:
novelcompound_df = pd.read_csv(
   DATA / "generated_part8.csv", index_col=0,
)

print("Shape of dataframe : ", novelcompound_df.shape)
novelcompound_df.head()

List with SMILES

In [ ]:
novelcompound_df_smiles = novelcompound_df.SMILES.tolist()

Convert SMILES to morgan fingerprints with radius 3. Use these fingerprints to predict the pIC50 of these compounds with the trained random forest regression model. 

In [ ]:
fps = []

for smiles in novelcompound_df_smiles: 
    fp = smiles_to_fp(smiles,'morgan3')
    fps.append(fp)

predictions = trained_model.predict(fps)

Show results

In [ ]:
novelcompound_df_smiles_results = pd.DataFrame({
"SMILES" : novelcompound_df_smiles,
"predicted_pIC50" : predictions
})

In [ ]:
novelcompound_df_smiles_results_m = pd.merge(
    novelcompound_df_smiles_results,
    novelcompound_df,
    on = "SMILES",
)

PandasTools.AddMoleculeColumnToFrame(novelcompound_df_smiles_results_m, smilesCol="SMILES")
novelcompound_df_smiles_results_m.sort_values(by="predicted_pIC50", ascending = False, inplace = True)
novelcompound_df_smiles_results_m.reset_index(drop=True, inplace=True)
novelcompound_df_smiles_results_m.head(10)

[Say something about the results, is the pIC50 high? Do they fulfill Ro5? How do they look? Scaffold? Bestaan ze al? Zijn ze te synthesiseren?]

Save results of the novel compounds

In [ ]:
output_df = novelcompound_df_smiles_results_m.drop("ROMol", axis = 1)
output_df.to_csv(DATA/"nieuwmoleculenlijst_voorspellingen.csv")
output_df.head()